In [39]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *
import pandas as pd 

no_of_reviews = 1000
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
driver = webdriver.Chrome(r"chromedriver.exe")
wait = WebDriverWait( driver, 10 )

# Append your app store urls here
url = "https://www.capterra.com/p/211162/Dropbox-Business/reviews/"

driver.get(url)

page = driver.page_source

soup_expatistan = BeautifulSoup(page, "html.parser")

expatistan_table = soup_expatistan.find("h1", class_="nb-type-xl")

print("App name: ", expatistan_table.string)

expand_pages = soup_expatistan.findAll("div", class_="nb-p-xl")
counter = 1

list_of_comments = []
for expand_page in expand_pages:
    try:
        print("\n===========\n")
        print("review：" + str(counter))
        print("Title: ", str(expand_page.find("div", class_="nb-type-lg-bold").text))

        allopinion = expand_page.findAll("div", class_="nb-text-md nb-my-xl")
        opinion_store=[]
        for opinion in allopinion:
            opinion_store.append(str(opinion.text))
        print(opinion_store)

        Overall = ''
        Pros = ''
        Cons = ''
        Switch = ''
        Switch_reason = ''
        Alter = ''
        Alter_reason = ''
        for opinion in opinion_store:
            if 'Overall:' in opinion:
                Overall = opinion
            elif 'Pros:' in opinion:
                Pros = opinion
            elif 'Cons:' in opinion:
                Cons = opinion
            elif 'Switched From:' in opinion:
                Switch = opinion
            elif 'Reasons for Switching to Dropbox Business:' in opinion:
                Switch_reason = opinion
            elif 'Alternatives Considered:' in opinion:
                Alter = opinion
            elif 'Reasons for Choosing Dropbox Business:' in opinion:
                Alter_reason = opinion
                
        #寫入DataFrame
        ndf = pd.DataFrame([{'Review':str(counter),
                     'Title':str(expand_page.find("div", class_="nb-type-lg-bold").text),
                     'Overall':Overall,
                     'Pros':Pros,
                     'Cons':Cons,
                     'Switch':Switch,
                     'Switch_reason':Switch_reason,
                     'Alter':Alter,
                     'Alter_reason':Alter_reason}],
                    columns = ['Review', 'Title', 'Overall', 'Pros', 'Cons', 'Switch', 'Switch_reason', 'Alter', 'Alter_reason'])
        list_of_comments.append(ndf)
        counter+=1
    except:
        pass

App name:  Dropbox Business Reviews


review：1
Title:  “Must use storage option for official works.”
['Overall:\xa0I mainly use Dropbox for my office work, yo store all the docs in cloud as backup. Dropbox provide me choice to share files with specific user with full privacy option. Uploading is so fast and for larger files , sometimes takes much times.\nBesides it provides 3rd party integration which gives us chance to accelarate our work.', 'Pros:\xa0Most important features Dropbox provides from my perspective is the automatic file backup option. Dropbox takes backup all my documents, helps me to work without fear of losing them. Beside dropbox have world-class security to protect our data..\nI can store all my data in cloud and share to official documents for work purpose very easily. Organizations of my documents is very good in Dropbox business. Moreover uploading speed is very fast , I can able to store big files in cloud storage easily and able to track from anywhere.', "Cons:\x

['Overall:\xa0my overall experience has been positive. there has been some issues as with any service, but dropbox has been great to work with and has been helpful.\n\nAdding the FTP capability a couple of years ago has been a huge upgrade to the account and makes me want to stick with them indefinitely', "Pros:\xa0Dropbox has been my go-to software for over 5 years. We use dropbox to collaborate between video projects, graphic projects, and photo projects. We have editors all over the world and nothing works better than dropbox. I've tried using google drive multiple times, but dropbox's desktop sync has been the best thing for our collaboration.\n\nTransferring files to clients have been so simple on both ends. Every time we've sent files using google drive, we've had nothing but problems.\n\nDropbox makes it super easy to send files and to gather files.\n\nTheir file request functionality has been amazing to work with and makes it easy to collaborate with clients outside of our drop

In [40]:
comments = pd.concat(list_of_comments, ignore_index=True)
comments.to_csv('Capterra_comments.csv', encoding='utf_8_sig')